크롤링 target URL : https://www.premierleague.com/clubs

In [2]:
import scrapy
import requests
from scrapy.http import TextResponse

step 1) crawler project 실행

In [ ]:
!scrapy startproject premierleague

In [ ]:
!tree premierleague

step 2) 구단별 URL link 수집 코드 작성

In [5]:
req = requests.get('https://www.premierleague.com/clubs')
response = TextResponse(req.url, body=req.text, encoding='utf-8')
club_links = response.xpath('//*[@id="mainContent"]/div/div/div/div[1]/div/ul/li/a/@href').extract()

len(club_links)
club_links

['/clubs/1/Arsenal/overview',
 '/clubs/127/Bournemouth/overview',
 '/clubs/131/Brighton-and-Hove-Albion/overview',
 '/clubs/43/Burnley/overview',
 '/clubs/46/Cardiff-City/overview',
 '/clubs/4/Chelsea/overview',
 '/clubs/6/Crystal-Palace/overview',
 '/clubs/7/Everton/overview',
 '/clubs/34/Fulham/overview',
 '/clubs/159/Huddersfield-Town/overview',
 '/clubs/26/Leicester-City/overview',
 '/clubs/10/Liverpool/overview',
 '/clubs/11/Manchester-City/overview',
 '/clubs/12/Manchester-United/overview',
 '/clubs/23/Newcastle-United/overview',
 '/clubs/20/Southampton/overview',
 '/clubs/21/Tottenham-Hotspur/overview',
 '/clubs/33/Watford/overview',
 '/clubs/25/West-Ham-United/overview',
 '/clubs/38/Wolverhampton-Wanderers/overview']

In [6]:
for club_link in club_links:
    club_link = 'https://www.premierleague.com' + club_link[:-8] + 'squad'
    print(club_link)

https://www.premierleague.com/clubs/1/Arsenal/squad
https://www.premierleague.com/clubs/127/Bournemouth/squad
https://www.premierleague.com/clubs/131/Brighton-and-Hove-Albion/squad
https://www.premierleague.com/clubs/43/Burnley/squad
https://www.premierleague.com/clubs/46/Cardiff-City/squad
https://www.premierleague.com/clubs/4/Chelsea/squad
https://www.premierleague.com/clubs/6/Crystal-Palace/squad
https://www.premierleague.com/clubs/7/Everton/squad
https://www.premierleague.com/clubs/34/Fulham/squad
https://www.premierleague.com/clubs/159/Huddersfield-Town/squad
https://www.premierleague.com/clubs/26/Leicester-City/squad
https://www.premierleague.com/clubs/10/Liverpool/squad
https://www.premierleague.com/clubs/11/Manchester-City/squad
https://www.premierleague.com/clubs/12/Manchester-United/squad
https://www.premierleague.com/clubs/23/Newcastle-United/squad
https://www.premierleague.com/clubs/20/Southampton/squad
https://www.premierleague.com/clubs/21/Tottenham-Hotspur/squad
https://

step 3) 선수별 URL link 수집 코드작성

In [7]:
player_links = []

for club_link in club_links:
    club_link = 'https://www.premierleague.com' + club_link[:-8] + 'squad'
    req = requests.get(club_link)
    response = TextResponse(req.url, body=req.text, encoding='utf-8')
    player_link = response.xpath('//*[@id="mainContent"]/div[2]/div/ul/li/a/@href').extract()
    for link in player_link:
        player_links.append(link)

len(player_links)

584

In [8]:
player_stat = []


for player_link in player_links:
    player_link = 'https://www.premierleague.com' + player_link[:-8] +'stats?co=1&se=210'
    player_stat.append(player_link)
    
print(len(player_stat))
player_stat

584


['https://www.premierleague.com/players/2651/Petr-Cech/stats?co=1&se=210',
 'https://www.premierleague.com/players/4985/Bernd-Leno/stats?co=1&se=210',
 'https://www.premierleague.com/players/4245/Emiliano-Martínez/stats?co=1&se=210',
 'https://www.premierleague.com/players/10542/Dejan-Iliev/stats?co=1&se=210',
 'https://www.premierleague.com/players/4474/Héctor-Bellerín/stats?co=1&se=210',
 'https://www.premierleague.com/players/4030/Laurent-Koscielny/stats?co=1&se=210',
 'https://www.premierleague.com/players/11575/Rob-Holding/stats?co=1&se=210',
 'https://www.premierleague.com/players/4472/Nacho-Monreal/stats?co=1&se=210',
 'https://www.premierleague.com/players/3869/Shkodran-Mustafi/stats?co=1&se=210',
 'https://www.premierleague.com/players/32638/Konstantinos-Mavropanos/stats?co=1&se=210',
 'https://www.premierleague.com/players/5368/Sead-Kolasinac/stats?co=1&se=210',
 'https://www.premierleague.com/players/5520/Stephan-Lichtsteiner/stats?co=1&se=210',
 'https://www.premierleague.c

step 4) 선수별 각종정보 수집 코드작성

In [9]:
# 상세 페이지 접속
req = requests.get('https://www.premierleague.com/players/3390/Dan-Gosling/stats?co=1&se=210')
response = TextResponse(req.url, body=req.text, encoding='utf-8')

In [10]:
# 이름
name = response.xpath('//*[@class="playerDetails"]/h1/div/text()').extract()[0]
name

'Dan Gosling'

In [11]:
# 소속팀
team = response.xpath('//*[@id="mainContent"]/div[2]/nav/div/section[1]/div[2]/a/text()').extract()
team[0]

'AFC Bournemouth'

In [12]:
# 포지션
position = response.xpath('//*[@id="mainContent"]/div[2]/nav/div/section[1]/div[4]/text()').extract()
position[0]

'Midfielder'

In [13]:
appearances = response.xpath('//*[@data-stat="appearances"]/text()').extract()[0].strip()
appearances

'147'

In [14]:
team_assists = response.xpath('//*[@id="mainContent"]/div[2]/div/div/div/div/div/ul/li[2]/div/div[2]/span/span/text()').extract()[0].strip()
team_assists

'2'

step 5) scrapy 툴 적용을 위한 step1) ~ step4) 코드 정리

- crawler/items.py

```
import scrapy

class CrawlerItem(scrapy.Item):
    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    discount_rate = scrapy.Field()
```

In [15]:
%%writefile premierleague/premierleague/items.py
import scrapy

class CrawlerItem(scrapy.Item):
    name = scrapy.Field()
    team = scrapy.Field()
    position = scrapy.Field()
    appearances = scrapy.Field()
    team_assists = scrapy.Field()

Overwriting premierleague/premierleague/items.py


In [16]:
%%writefile premierleague/premierleague/spiders/spider.py
import scrapy
import requests
from scrapy.http import TextResponse
from premierleague.items import CrawlerItem

# start_urls -> parse -> parse_page_contents 순으로 호출
class Spider(scrapy.Spider):
    # 스파이더의 이름
    name = "premierleague"
    allow_domain = ["https://www.premierleague.com"]
    # 크롤링의 시작 URL
    start_urls = ["https://www.premierleague.com/clubs"]

    # link 리스트를 가져옴
    def parse(self, response):
    
        player_links = []
        req = requests.get('https://www.premierleague.com/clubs')
        response = TextResponse(req.url, body=req.text, encoding='utf-8')
        club_links = response.xpath('//*[@id="mainContent"]/div/div/div/div[1]/div/ul/li/a/@href').extract()

        for club_link in club_links:
            club_link = 'https://www.premierleague.com' + club_link[:-8] + 'squad'
            req = requests.get(club_link)
            response = TextResponse(req.url, body=req.text, encoding='utf-8')
            player_link = response.xpath('//*[@id="mainContent"]/div[2]/div/ul/li/a/@href').extract()
            for link in player_link:
                player_links.append(link)        
                
        player_stat_links = []

        for player_link in player_links:
            player_link = 'https://www.premierleague.com' + player_link[:-8] +'stats?co=1&se=210'
            player_stat_links.append(player_link)

        for link in player_stat_links[:10]:
            yield scrapy.Request(link, callback=self.parse_page_contents)

    # 각페이지의 link로 접속하여 데이터를 가져옴
    def parse_page_contents(self, response):
        item = CrawlerItem()
        item["name"] = response.xpath('//*[@class="playerDetails"]/h1/div/text()').extract()[0]
        item["team"] = response.xpath('//*[@id="mainContent"]/div[2]/nav/div/section[1]/div[2]/a/text()').extract()[0]
        item["position"] = response.xpath('//*[@id="mainContent"]/div[2]/nav/div/section[1]/div[4]/text()').extract()[0]
        item["appearances"] = response.xpath('//*[@data-stat="appearances"]/text()').extract()[0].strip()
        item["team_assists"] = response.xpath('//*[@id="mainContent"]/div[2]/div/div/div/div/div/ul/li[2]/div/div[2]/span/span/text()').extract()[0].strip()
        
        yield item

Overwriting premierleague/premierleague/spiders/spider.py


step 5) scrapy 툴 적용

1) scrapy.cfg 파일이 있는 디렉토리에서 아래의 명령을 실행하면 크롤링이 시작됨

$ scrapy crawl premierleague

2) 결과를 csv 파일로 저장
$ scrapy crawl premierleague -o premierleague.csv

In [18]:
df = pd.read_csv("premierleague/premierleague.csv")
df

,appearances,name,position,team,team_assists
0,443,Petr Cech,Goalkeeper,Arsenal,202
1,239,Laurent Koscielny,Defender,Arsenal,2
2,31,Rob Holding,Defender,Arsenal,0
3,172,Nacho Monreal,Defender,Arsenal,12
4,141,Héctor Bellerín,Defender,Arsenal,17
5,3,Konstantinos Mavropanos,Defender,Arsenal,0
6,0,Dejan Iliev,Goalkeeper,Arsenal,0
7,69,Shkodran Mustafi,Defender,Arsenal,3
8,12,Bernd Leno,Goalkeeper,Arsenal,1
9,6,Emiliano Martínez,Goalkeeper,Arsenal,3


In [ ]:
max(df['appearances'])

In [ ]:
df['team'].value_counts()

In [ ]:
df[(df['appearances'] > 100) & (df['team'] == 'Manchester United')]

step 6) 파이프라인 구현

- 결과를 깔끔하게 정리해서 저장


- spider의 items 데이터가 pipelines에 정의된 대로 저장


- pipelines.py 

```
%%writefile premierleague/premierleague/pipelines.py
import csv

class CrawlerPipeline(object):

    def __init__(self):
        self.csvwriter = csv.writer(open("premierleague.csv", "w"))
        self.csvwriter.writerow(["name","team","position","appearances",'team_assists'])

    def process_item(self, item, spider):
        row = []
        row.append(item["name"])
        row.append(item["team"])
        row.append(item["position"])
        row.append(item["appearances"])
        row.append(item["team_assists"])
        self.csvwriter.writerow(row)
        return item
```

In [ ]:
%%writefile premierleague/premierleague/pipelines.py
import csv

class CrawlerPipeline(object):

    def __init__(self):
        self.csvwriter = csv.writer(open("premierleague.csv", "w"))
        self.csvwriter.writerow(["name","team","position","appearances",'team_assists'])

    def process_item(self, item, spider):
        row = []
        row.append(item["name"])
        row.append(item["team"])
        row.append(item["position"])
        row.append(item["appearances"])
        row.append(item["team_assists"])
        self.csvwriter.writerow(row)
        return item

In [ ]:
!cat crawler/crawler/pipelines.py